In [1]:
import pandas as pd
import numpy as np
import uuid
import json

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/borja/Documents/MBIT/30_TFM/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/var/folders/2y/86y88mmd2x9cnkf1hhh3q8l00000gn/T/ipykernel_26681/1424932742.py", line 1, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/borja/Documents/MBIT/30_TFM/venv/lib/python3.11/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/borja/Documents/MBIT/30_TFM/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
  File "/Users/borja/Documents/MBIT/30_TFM/venv/lib/python3.11/site-packages/IPython/core/ultratb.py"

In [ ]:
DATA = "https://raw.githubusercontent.com/borjaregueral/route_optimizer_platform/main/data_sources_generator/synthetic_data/results/clients_geo.csv"
max_weight_per_order = 50
max_vol_per_order = 100
status = "active"

In [ ]:
def generate_orders(data, status, max_vol_per_order, max_weight_per_order):
    # Filter for clients with the specified status
    filtered_data = data[data['status'] == status]
    
    # Generate orders using list comprehension
    orders_list = [
        {
            'client_id': client_id, 
            'order_id': str(uuid.uuid4()),  # Generate a unique order_id
            'volume': np.random.uniform(1, max_vol_per_order),  # Randomly generate volume
            'weight': np.random.uniform(1, max_weight_per_order),  # Randomly generate weight
            'n_objects': np.random.randint(1, 10)  # Randomly generate number of objects
        } 
        for client_id in filtered_data['client_id']
    ]

    # Convert the list of dictionaries to a DataFrame
    orders = pd.DataFrame(orders_list)
    
    return orders

In [ ]:
def transform_to_json(clients):
    """
    Transforms a DataFrame of client data into a JSON string with a nested structure.

    Parameters:
    - clients: pandas.DataFrame

    Returns:
    - str: A JSON string representation of the transformed client data.
    """
    clients_final = [
        {
            'client_id': row['client_id'],
            'location': { 
                'address': row['address'],
                'lat': row['lat'],
                'lon': row['lon']
            },
            'order_id': row['order_id'],
            'order': { 
                'n_objects': row['n_objects'],
                'volume': row['volume'],
                'weight': row['weight']
            },
            'status': row['status']
        }
        for _, row in clients.iterrows()
    ]

    # Convert to JSON and return the string
    return json.dumps(clients_final, indent=4)


In [ ]:
# Load data
users = pd.read_csv(DATA)

# Clean dataset keeping only relevant data
users_clean = (
    users
    .loc[:, ["client_id", "address", "status", "points","lat", "lon"]]
)

In [ ]:
# Generate and store orders from Active clients
orders = generate_orders(users_clean, status, max_vol_per_order, max_weight_per_order)

# Perform an inner join between orders and users_clean DataFrames
clients = pd.merge(orders, users_clean, left_on='client_id', right_on='client_id', how='inner')

In [ ]:
# Create a new DataFrame with nested structure
clients_final = transform_to_json(clients)

# Print the final JSON string
print(clients_final)

[
    {
        "client_id": 298104694689,
        "location": {
            "address": "Bypass Sur",
            "lat": 40.390130799999994,
            "lon": -3.683518876417766
        },
        "order_id": "498b821e-a8c1-4876-8ecb-05912791d902",
        "order": {
            "n_objects": 5,
            "volume": 80.57954913557874,
            "weight": 16.296355287733935
        },
        "status": "active"
    },
    {
        "client_id": 484625614518,
        "location": {
            "address": "Calle de Embajadores188",
            "lat": 40.39298055,
            "lon": -3.6942722908057015
        },
        "order_id": "d923d690-e053-41e8-bfe4-910130a40551",
        "order": {
            "n_objects": 5,
            "volume": 82.48537005761371,
            "weight": 36.182779538246386
        },
        "status": "active"
    },
    {
        "client_id": 355616727318,
        "location": {
            "address": "Calle de Antonio Nebrija",
            "lat": 40.4036985,
  

In [ ]:
# Save the JSON string to a file
with open('clients_final.json', 'w') as file:
    file.write(clients_final)

# Print a confirmation message
print("Se ha guardado 'clients_final.json'")

Se ha guardado 'clients_final.json'
